In [2]:
import atexit, threading, json
from http.server import BaseHTTPRequestHandler, HTTPServer
import urllib.parse
import onem2mlib
import onem2mlib.exceptions as EXC
import onem2mlib.constants as CON
import onem2mlib.internal as INT
import requests
import uuid

In [3]:
# Encryption
import re
import base64
from base64 import b64decode
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
import hmac
import hashlib

In [4]:
def hash_hmac256(message):
    API_SECRET = "ESWTEAM9"

    signature = hmac.new(
            bytes(API_SECRET , 'ascii'),
            msg = bytes(message , 'ascii'),
            digestmod = hashlib.sha256
        ).hexdigest()
    return signature

print(hash_hmac256("Iaa0aBgnZEmfdBEr21orLHbWhd4ySNJcVa49NvuOf/3LWefa4Qr/FInjYZWlOuR7"))

48673d0b9e3a417dec5485496bac1fa414a4e57d5b61219fc4a9fd5ac1a231db


In [5]:
def decode_base64(data, altchars=b'+/'):
    data = re.sub(rb'[^a-zA-Z0-9%s]+' % altchars, b'', data)
    missing_padding = len(data) % 4
    if missing_padding:
        data += b'='* (4 - missing_padding)
    return base64.b64decode(data, altchars)

def decode_AES(ciphertext):
    key = "2B7E151628AED2A6ABF7158809CF4F3C"
    iv = "AAAAAAAAAAAAAAAAAAAAAA=="

    key = bytearray.fromhex(key)
    iv = b64decode(iv).hex()
    iv = bytearray.fromhex(iv)
    ciphertext = b64decode(ciphertext)

    cipher = AES.new(key, AES.MODE_CBC, iv)
    decrypted = cipher.decrypt(ciphertext)
    decryptedUnpad = unpad(decrypted, AES.block_size)
    return decode_base64(decryptedUnpad).decode("utf-8")

In [ ]:
decode_AES('IeuAnZdknufs4a5NL09+u0fBi/66Ro7U+4vhyURAE7rZB1V6a2CMmZ3A6XsO8Akm')

In [15]:
def print_content(content):
    content = content.split(',')
    unique_id, feature1, feature2, feature3, grid, time_stamp =  content[0], content[1], content[2], content[3], content[4], content[5]

    # df = pd.read_csv('dataset.csv')
    # display(df)
    data_point = {
        "unique_id": unique_id,
        "feature_1": feature1,
        "feature_2": feature2,
        "feature_3": feature3,
        "time_stamp": time_stamp,
        "grid": grid
    }
    print(data_point)

acxyy8GxS1ousrQe/wDHtRtHgIT6C/CYt6u7I+asJclCIF+4OaSzawfYMdEliGkf3Zymw1LZ8/0Yhswe6mCnOq/8uznlbqzSarLjGiTmDY5qmZTecO9jlyDM8eNgMt84Ve6fOKLTK8KSxxpUOFFIJhV3J/s74vC8k6RCHM1u7Y6HYUFepu2C8blyL3Vi3OtS,501e397ad9083704a5d9e766e6e41d915969d427f010ceb9de9c88e5b2838aa8
-----------
10000019,0,0,0,0000000000000000000000000000000000000000000000000000000000000000,2021-11-13T12:47:55Z 501e397ad9083704a5d9e766e6e41d915969d427f010ceb9de9c88e5b2838aa8
501e397ad9083704a5d9e766e6e41d915969d427f010ceb9de9c88e5b2838aa8
---------------
Hash verified. Successful transfer.
10000019,0,0,0,0000000000000000000000000000000000000000000000000000000000000000,2021-11-13T12:47:55Z
{'unique_id': '10000019', 'feature_1': '0', 'feature_2': '0', 'feature_3': '0', 'time_stamp': '2021-11-13T12:47:55Z', 'grid': '0000000000000000000000000000000000000000000000000000000000000000'}
mVDpJ/s3dCpFAScbi3V8wh4pjlEw09oZCiwFoDtEH+7nJiDTbt5DHvkaT2ioSyEh62jSn1XP/Asa05BGS54QDyuVjuWhQWn9p9kN5R85Uy3YxLAWgjClc4+GTnPUMZB4KwCTubR3y6WVvJLwXR8zKrY0

In [ ]:
hash_hmac256("FnhNls08YIWZKy9Z0P5WkV45ylYMnixKX+73VWpHnPdbHzCQjwaV88Ti0Wuo/WH1")

In [7]:
def verify_hash(to_decode, to_hash):
    to_decode = decode_AES(to_decode)
    calc_hash = hash_hmac256(to_decode)
    print("-----------")
    print(to_decode, to_hash)
    print(calc_hash)
    print("---------------")
    if calc_hash == to_hash:
        print("Hash verified. Successful transfer.")
    else:
        print("Hash doesn't match. Something fishy.")

In [8]:
_server = None
_thread = None
_port = 1401


def call_back(res):
    print(res.content)
    tup = res.content.split(',')
    to_decode, to_hash = tup[0], tup[1]
    verify_hash(to_decode, to_hash)
    dec_content = decode_AES(to_decode)
    print(dec_content)
    print_content(dec_content)

# Start the notification server in a background threa
def _startNotificationServer():
	global _server, _thread
	if _thread:
		return
	# Start processing requests in a separate thread.
	# Listen on any interface/IP address.
	_server = HTTPNotificationServer(('', _port), HTTPNotificationHandler)
	_thread = threading.Thread(target=_server.run)
	_thread.start()

# Stop the thread/notification server
@atexit.register
def _stopNotificationServer():
	global _server, _thread
	if not _server or not _thread:
		return
	# Shutdown server
	_server.shutdown()
	_thread.join()
	_server = None
	_thread = None

# This class implements the notification server that runs in the background.
class HTTPNotificationServer(HTTPServer):
	def run(self):
		try:
			self.serve_forever()
		finally:
			# Clean-up server (close socket, etc.)
			self.server_close()

# This class implements the handler that reseives the requests
class HTTPNotificationHandler(BaseHTTPRequestHandler):
	# Handle incoming notifications (POST requests)
	def do_POST(self):
			# Construct return header
			self.send_response(200)
			self.send_header('X-M2M-RSC', '2000')
			self.end_headers()

			# Get headers and content data
			length = int(self.headers['Content-Length'])
			contentType = self.headers['Content-Type']
			post_data = self.rfile.read(length)
			#print(post_data)
			threading.Thread(target=self._handleJSON(post_data), args=(post_data)).start()
			

	# Catch and ignore all log messages
	def log_message(self, format, *args):
		return
	

	# Handle JSON notifications 
	def _handleJSON(self, data):
		jsn =  json.loads(data.decode('utf-8'))
		#print(jsn)
		# check verification request
		vrq = INT.getALLSubElementsJSON(jsn, 'vrq')
		if len(vrq) == 0:										
			vrq = INT.getALLSubElementsJSON(jsn, 'm2m:vrq')
		if len(vrq) > 0 and vrq[0] == True:
			return 	# do nothing

		# get the sur first
		sur = INT.getALLSubElementsJSON(jsn, 'sur')
		if len(sur) == 0:										
			sur = INT.getALLSubElementsJSON(jsn, 'm2m:sur')
		if len(sur) > 0:
			sur = sur[0]
		else:
			return 	# must have a subscription ID

		# get resource
		rep = INT.getALLSubElementsJSON(jsn, 'rep')
		if len(rep) == 0:							
			rep = INT.getALLSubElementsJSON(jsn, 'm2m:rep')
		if len(rep) > 0:
			jsn = rep[0]
			type = INT.getALLSubElementsJSON(jsn, 'ty')
			if type and len(type) > 0:
				resource = INT._newResourceFromType(type[0], None)
				resource._parseJSON(jsn)
				call_back(resource)
###############################################################################

In [9]:
_startNotificationServer()

In [12]:
def subscribe():
    global _port
    headers = {
        'X-M2M-Origin': 'rTI9uxe@NN:7rn5sOIaYf',
        'Content-type': 'application/json;ty=23'
     }
    
    body = {
        "m2m:sub":{
            "rn":str(uuid.uuid4()),
            "nu":[ f"http://10.11.0.150:{_port}/" ],
            "nct":1
        }
    }
    try:
        response = requests.post("http://esw-onem2m.iiit.ac.in:443/~/in-cse/in-name/Team-9/testing/", json=body, headers=headers)
    except TypeError:
        response = requests.post("http://esw-onem2m.iiit.ac.in:443/~/in-cse/in-name/Team-9/testing/", data=json.dumps(body), headers=headers)
    print('Return code : {}'.format(response.status_code))
    print('Return Content : {}'.format(response.text))
    return
    

In [13]:
subscribe()

Return code : 201
Return Content : {
   "m2m:sub" : {
      "rn" : "29c77bea-a606-4a59-af23-84a53b78796d",
      "ty" : 23,
      "ri" : "/in-cse/sub-637089926",
      "pi" : "/in-cse/cnt-234319241",
      "ct" : "20211113T124036",
      "lt" : "20211113T124036",
      "acpi" : [ "/in-cse/acp-516227225", "/in-cse/acp-648169147" ],
      "nu" : [ "http://10.11.0.150:1401/" ],
      "nct" : 1
   }
}
KU3lbwzjLlR3SrJsip2AS/Y+6rEU1cWIDVpOu1C9uhhinDMOcy6UxVyKu9UfwsBhnUlihTGrU5Kbzi+L54x6XQm16DIS6Y9oen7LKBD2Wgtk0u/0tAnLtaZ4O98g5ZrUAnAO8RVxb0I3q3f9lXFW/0ZIyCzEOwqKMS0a8U/2b2hcJ+j3cjW4cWgxR6bifBkk,b105ed8196caa6c28f5b9e86f61bc024b0c56440f43b84cedc318a97c2d6bbee
-----------
10000000,0,0,0,0000000000000000000000000000000000000000000000000000000000000000,2021-11-13T12:44:43Z b105ed8196caa6c28f5b9e86f61bc024b0c56440f43b84cedc318a97c2d6bbee
b105ed8196caa6c28f5b9e86f61bc024b0c56440f43b84cedc318a97c2d6bbee
---------------
Hash verified. Successful transfer.
10000000,0,0,0,0000000000000000000000000000000

In [ ]:
_stopNotificationServer()

In [ ]:
def get_data():
    headers = {
        'X-M2M-Origin': 'rTI9uxe@NN:7rn5sOIaYf',
        'Content-type': 'application/json'
     }

    response = requests.get("http://esw-onem2m.iiit.ac.in:443/~/in-cse/in-name/Team-9/testing?rcn=4", headers=headers)
    print('Return code : {}'.format(response.status_code))
    print('Return Content : {}'.format(response.text))
    _resp = json.loads(response.text)
    return response.status_code , _resp

In [ ]:
i , j = get_data()